In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from numpy import random
from CFN_impl import CFNCell

In [2]:
events_df = pd.read_csv('./data/retailrocket/events.csv')
category_tree_df = pd.read_csv('./data/retailrocket/category_tree.csv')
item_properties_1_df = pd.read_csv('./data/retailrocket/item_properties_part1.csv')
item_properties_2_df = pd.read_csv('./data/retailrocket/item_properties_part2.csv')

In [3]:
item_properties_df = pd.concat([item_properties_1_df, item_properties_2_df])

In [4]:
item_properties_df

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513
...,...,...,...,...
9275898,1433646000000,236931,929,n12.000
9275899,1440903600000,455746,6,150169 639134
9275900,1439694000000,347565,686,610834
9275901,1433646000000,287231,867,769062


In [5]:
events_df

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN
2756097,1438399813142,762376,view,115946,NaN
2756098,1438397820527,1251746,view,78144,NaN
2756099,1438398530703,1184451,view,283392,NaN


In [6]:
events_df = events_df.drop(['event','transactionid'],axis=1)

不重复的用户数量

In [7]:
events_df.visitorid.nunique()

1407580

将目标放在活动交互次数大于10次的用户

In [8]:
# users = events_df['visitorid']
# users = users.value_counts()
# users = users.iloc[users.values > 10].index
# users = users.sort_values()
# users

In [9]:
dataset = pd.read_csv('./data/retailrocket/SLdata.csv')

In [10]:
group = dataset.groupby('visitorid',group_keys=False).groups

做出每个用户访问物品的列表

In [11]:
user_list=[]
action_list=[]
for user in dataset.visitorid.unique().tolist():
    item_list=[]
    for item in group[user]:
        item = dataset['itemid'][item]
        item_list.append(item)
    user_list.append(user)
    action_list.append(item_list)

In [12]:
data = {'user':user_list, 'actions':action_list}
df = pd.DataFrame(data)

In [13]:
#df.to_csv('./data/retailrocket/seqData.csv',index=False)

In [14]:
def seq_data_iter_random(corpus, batch_size, num_steps):  #@save
    """使用随机抽样生成一个小批量子序列"""
    # 减去1，是因为我们需要考虑标签
    num_subseqs = (len(corpus) - 1) // num_steps
    # 长度为num_steps的子序列的起始索引
    initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
    # 在随机抽样的迭代过程中，
    # 来自两个相邻的、随机的、小批量中的子序列不一定在原始序列上相邻
    random.shuffle(initial_indices)

    def data(pos):
        # 返回从pos位置开始的长度为num_steps的序列
        return corpus[pos: pos + num_steps]

    num_batches = num_subseqs // batch_size
    for i in range(0, batch_size * num_batches, batch_size):
        # 在这里，initial_indices包含子序列的随机起始索引
        initial_indices_per_batch = initial_indices[i: i + batch_size]
        X = [data(j) for j in initial_indices_per_batch]
        Y = [data(j + 1) for j in initial_indices_per_batch]
        yield tf.constant(X), tf.constant(Y)
        #yield X, Y

In [15]:
def dataset_loader(df, batch_size, num_steps):
    for index in df.index:
        seq_data = df.loc[index].actions
        x_list=[]
        y_list=[]
        for X,Y in seq_data_iter_random(seq_data, batch_size, num_steps):
            x_list.append(X)
            y_list.append(Y)
        yield index, [x_list, y_list]        

In [16]:
def dataset_loader(df, batch_size, num_steps):
    for index in df.index:
        seq_data = df.loc[index].actions
        action_and_label = seq_data_iter_random(seq_data, batch_size, num_steps)
        yield index, action_and_label      

In [17]:
embed = tf.keras.layers.Embedding(
    input_dim=item_size, 
    output_dim=64,
    #embeddings_initializer=tf.keras.initializers.GlorotUniform(),
    input_length=1
)

NameError: name 'item_size' is not defined

In [ ]:
item_embedded = embed(item_properties_df.itemid.unique())

In [ ]:
lookup = tf.nn.embedding_lookup

# 数据处理

In [18]:
from DataHelper import Data

In [19]:
dataset = Data(1, 3)

In [20]:
item_size = dataset.item_size

# 训练过程

In [21]:
from d2l import tensorflow as d2l
import math
import pandas as pd
import numpy as np
import tensorflow as tf
from numpy import random
from CFN_impl import CFNCell

In [22]:
class RNNModel(tf.keras.Model):
    def __init__(self, rnn_layer, item_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.dense = tf.keras.layers.Dense(item_size)
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.input_embed = tf.keras.layers.Embedding(
            input_dim=item_size, output_dim=64
        )
        # self.output_embed = tf.keras.layers.Embedding(
        #     input_dim=256, output_dim=item_size
        # )
        # self.state_embed = tf.keras.layers.Embedding(
        #     input_dim=1, output_dim=32
        # )


    def call(self, inputs, state):
        X = self.input_embed(inputs)
        #X = inputs
        # rnn返回两个以上的值
        Y, *state = self.rnn(X, state)
        output_ = self.dropout(Y)
        output = self.dense(output_)
        #output = self.output_embed(output)
        return output, state

    def begin_state(self, *args, **kwargs):
        return self.rnn.cell.get_initial_state(*args, **kwargs)

In [23]:
cfn_cell = CFNCell(256)
cfn_layer = tf.keras.layers.RNN(cfn_cell,
                                return_state=True,
                                return_sequences=True)
net = RNNModel(cfn_layer, item_size)


2023-05-01 11:22:37.629289: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 11:22:37.668062: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 11:22:37.668148: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 11:22:37.668624: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [24]:
def grad_clipping(grads, theta):  #@save
    """裁剪梯度"""
    theta = tf.constant(theta, dtype=tf.float32)
    new_grad = []
    for grad in grads:
        if isinstance(grad, tf.IndexedSlices):
            new_grad.append(tf.convert_to_tensor(grad))
        else:
            new_grad.append(grad)
    norm = tf.math.sqrt(sum((tf.reduce_sum(grad ** 2)).numpy()
                        for grad in new_grad))
    norm = tf.cast(norm, tf.float32)
    if tf.greater(norm, theta):
        for i, grad in enumerate(new_grad):
            new_grad[i] = grad * theta / norm
    else:
        new_grad = new_grad
    return new_grad


In [25]:
#@save
def train_epoch_ch8(net, train_iter, loss, updater, item_size, use_random_iter):
    """训练模型一个迭代周期(定义见第8章)"""
    state = None
    metric = d2l.Accumulator(2)  # 训练损失之和,词元数量
    #print(next(train_iter))
    for X, Y in train_iter:
        #print('AAAAAAAAAAA!')
        #X = lookup(item_embedded, X)
        #Y = lookup(item_embedded, Y)
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], dtype=tf.float32)
        with tf.GradientTape(persistent=True) as g:
            y_hat, state = net(X, state)
            y = tf.one_hot(Y, item_size)
            l = loss(y, y_hat)
        params = net.trainable_variables
        grads = g.gradient(l, params)
        #grads = grad_clipping(grads, 1)
        updater.apply_gradients(zip(grads, params))
        # Keras默认返回一个批量中的平均损失
        metric.add(l , 1)
        #print('immediate loss:',l)
    return metric[0] / metric[1]

In [33]:
#@save
def train_ch8(net, dataset, lr, num_epochs, strategy,
              use_random_iter=False):
    """训练模型(定义见第8章)"""
    with strategy.scope():
        loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        updater = tf.keras.optimizers.SGD(lr)
    #animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
     #                      legend=['train'], xlim=[10, num_epochs])
    #predict = lambda prefix: predict_ch8(prefix, 50, net, vocab)
    # 训练和预测
    for epoch in range(num_epochs):
        df = dataset.df
        train_iter = dataset.dataset_loader(df, 1, 3)
        item_size = dataset.item_size
        metric = d2l.Accumulator(2) 
        for user, actions in train_iter:
            avg_loss = train_epoch_ch8(net, actions, loss, updater, item_size,
                                        use_random_iter)
            #print('userid:',user, 'avg_loss:',avg_loss)
            metric.add(avg_loss , 1)
            if metric[1] % 1000 == 0:
                print('epoch:',epoch, 'metric0:',metric[0],'\nmetric1:',metric[1], 'loss:', metric[0]/metric[1])
        # if (epoch + 1) % 10 == 0:
        #     #print(predict('time traveller'))
        #animator.add(epoch, [metric[0] / metric[1]])
        #device = d2l.try_gpu()._device_name
        #print(f'avg_loss {avg_loss:.1f}, {speed:.1f} 词元/秒 {str(device)}')
    #print(predict('time traveller'))
    #print(predict('traveller'))


In [27]:
device_name = d2l.try_gpu()._device_name
strategy = tf.distribute.OneDeviceStrategy(device_name)

In [34]:
train_ch8(net, dataset, 1, 10, strategy, True)

epoch: 0 metric0: 80.66035545887623 
metric1: 1000.0 loss: 0.08066035545887623
epoch: 0 metric0: 161.21433961096824 
metric1: 2000.0 loss: 0.08060716980548412
epoch: 0 metric0: 241.65668787058254 
metric1: 3000.0 loss: 0.08055222929019418
epoch: 0 metric0: 321.9673492880946 
metric1: 4000.0 loss: 0.08049183732202365
epoch: 0 metric0: 402.12291196082526 
metric1: 5000.0 loss: 0.08042458239216506
epoch: 0 metric0: 482.1777900880955 
metric1: 6000.0 loss: 0.08036296501468258
epoch: 0 metric0: 562.1176471600505 
metric1: 7000.0 loss: 0.08030252102286436
epoch: 0 metric0: 641.9064472170838 
metric1: 8000.0 loss: 0.08023830590213547
epoch: 0 metric0: 721.5702013660066 
metric1: 9000.0 loss: 0.08017446681844517
epoch: 0 metric0: 801.1212357881288 
metric1: 10000.0 loss: 0.08011212357881288
epoch: 0 metric0: 880.5495746119254 
metric1: 11000.0 loss: 0.08004996132835686
epoch: 0 metric0: 959.8463301502162 
metric1: 12000.0 loss: 0.07998719417918468
epoch: 0 metric0: 1039.0362181892997 
metric1:

KeyboardInterrupt: 